In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import string
import nltk
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

In [2]:
df = pd.read_csv('Ournews')

In [3]:
df.drop(columns= ['Unnamed: 0', 'url', 'date_extracted', 'author'], inplace= True)

In [4]:
df

,source,title,content
0,aawsat,بريطانيا: أربعة محاور لاستراتيجية جديدة تتصدى ...,حدد رئيس الوزراء البريطاني ديفيد كاميرون، اليو...
1,aawsat,الرئيس هادي يعين نايف البكري محافظا لعدن,أصدر الرئيس عبدربه منصور هادي رئيس الجمهورية ا...
2,aawsat,مجلس الأمن يصادق على «الاتفاق النووي».. وإيران...,وافق مجلس الأمن الدولي، اليوم (الاثنين)، على ا...
3,aawsat,«نيران صديقة» أميركية تودي بحياة 14 جنديا أفغانيا,قال مسؤولون أفغان ان 14 جنديا أفغانيا قتلوا في...
4,aawsat,الخارجية الإيطالية: أربعة من مواطنينا مخطوفون ...,أعلنت وزارة الخارجية الايطالية، اليوم (الاثنين...
...,...,...,...
31025,was,عام / وزير الشؤون البلدية يوجه بإنشاء مكاتب لإ...,الرياض 26 شوال 1436 هـ الموافق 11 اغسطس 2015 ...
31026,was,عام / هيئة الإغاثة تصف أوضاع اللاجئين الهاربين...,جدة 26 شوال 1436 هـ الموافق 11 أغسطس 2015 م و...
31027,was,عام / المشرف العام على مركز الملك سلمان للإغاث...,الرياض 26 شوال 1436 هـ الموافق 11 أغسطس 2015 ...
31028,was,رياضي / أخضر الطائرة في مواجهة مرتقبة أمام الج...,الرياض 26 شوال 1436 هـ الموافق 11 أغسطس 2015 ...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31030 entries, 0 to 31029
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   source   31030 non-null  object
 1   title    30963 non-null  object
 2   content  30897 non-null  object
dtypes: object(3)
memory usage: 727.4+ KB


In [6]:
df.duplicated().sum()

604

In [7]:
df.drop_duplicates(inplace = True)

In [8]:
df.duplicated().sum()

0

In [9]:
max(df.content.str.len())

33687.0

In [10]:
min(df.content.str.len())

1.0

In [11]:
df.content.unique()

array(['حدد رئيس الوزراء البريطاني ديفيد كاميرون، اليوم (الاثنين)، ملامح استراتيجية للتصدي للتطرف داخل بريطانيا؛ وهي مسألة اعتبرها كاميرون "صراع جيلنا"، متعهدا خلال خطابه في مدينة بيرمنغهام بالتصدي لهؤلاء الذين ينشرون التطرف بين الشبان المسلمين البريطانيين.\n\n ورسم كاميرون الاطار العام لاستراتيجية مكافحة التطرف التي المقرر ان تنشر كاملة في وقت لاحق هذا العام، والتي تسعى للتصدي لانتشار الأفكار المتطرفة التي يروج لها متشددو تنظيم "داعش".\n\n وحسبما تناقلت وسائل الإعلام البريطانية، فإن خطة رئيس الوزراء ستكون على مدى خمسة أعوام للقضاء على التطرف الداخلي من خلال أربعة محاور، وهي: القضاء على إيديولوجية التطرف، والوقوف في وجه عمليات التجنيد وغسل الأدمغة بالأفكار المتطرفة، وإعادة اصوات الاسلام المعتدل إلى المجتمع البريطاني، والتعامل مع أزمات الهوية التي يواجهها بعض المسلمين المولودين في بريطانيا.\n\n وسيكون من الأهداف الرئيسية للاستراتيجية مكافحة صعود من يطلق عليهم اسم "متطرفي الداخل"، وهو أمر يقول كاميرون انه لا يمكن عمله دون فهم الاسباب التي تجتذب الناس لـ"داعش" والتصدي لها. إذ قال كاميرون 

In [12]:
df.content.isna().sum()

130

In [13]:
df= df.dropna(subset= ['content'])

In [14]:
df.content.isna().sum()

0

In [30]:
import re #For Regex

# adapted from https://github.com/bakrianoo/aravec
# function to clean and normalize text 
def clean_text(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']  
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
        
    text = text.strip()
    
    return text

In [31]:
data = df['content']

data['cleaned_1'] = data.apply(lambda x: clean_text(x))

In [32]:
data.head(20)

0     حدد رءيس الوزراء البريطاني ديفيد كاميرون اليوم...
1     اصدر الرءيس عبدربه منصور هادي رءيس الجمهوريه ا...
2     وافق مجلس الامن الدولي اليوم الاثنين علي اتفاق...
3     قال مسءولون افغان ان 4 جنديا افغانيا قتلوا حاد...
4     اعلنت وزاره الخارجيه الايطاليه اليوم الاثنين ا...
5     ارتكبت الميليشيات الحوثيه مجزره مدينه عدن جنوب...
6     تحد جديد للكونغرس الاميركي رفض الرءيس باراك او...
7     افاق سكان حي الشيخ رضوان وشارع النفق وسط مدينه...
8     تحول «فقهاء» وقاده تنظيم داعش ليبيا الي نوع جد...
9     وسط تفاقم ازمات الكهرباء والماء السودان لوحت ح...
10    عقد اعضاء الحكومه اليمنيه ومسءولون امنيون للمر...
11    سددت قوات الامن السعوديه ضربات قاصمه للارهاب ا...
12    صعد المرشد الاعلي الايراني ايه الله علي خامنءي...
13    شيع النظام السوري امس جثمان اللواء محسن مخلوف ...
14    تفحص السلطات الماليه الاميركيه سلوك عدد الشركا...
15    تحرير المقاومه المواليه للحكومه اليمنيه الشرعي...
16    يسعي المحققون الاميركيون لتحديد السبب دفع بالش...
17    خطوه عدها محللون محاوله خصوم الرءيس الامير

In [33]:

import string

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [34]:
#Remove Punctuation
data['cleaned_2'] = data['cleaned_1'].apply(lambda x: remove_punctuations(x))
data['cleaned_2'].head(10)

<ipython-input-34-26467622e5f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_2'] = data['cleaned_1'].apply(lambda x: remove_punctuations(x))


0    حدد رءيس الوزراء البريطاني ديفيد كاميرون اليوم...
1    اصدر الرءيس عبدربه منصور هادي رءيس الجمهوريه ا...
2    وافق مجلس الامن الدولي اليوم الاثنين علي اتفاق...
3    قال مسءولون افغان ان 4 جنديا افغانيا قتلوا حاد...
4    اعلنت وزاره الخارجيه الايطاليه اليوم الاثنين ا...
5    ارتكبت الميليشيات الحوثيه مجزره مدينه عدن جنوب...
6    تحد جديد للكونغرس الاميركي رفض الرءيس باراك او...
7    افاق سكان حي الشيخ رضوان وشارع النفق وسط مدينه...
8    تحول «فقهاء» وقاده تنظيم داعش ليبيا الي نوع جد...
9    وسط تفاقم ازمات الكهرباء والماء السودان لوحت ح...
Name: content, dtype: object

In [35]:
data['cleaned_3'] = data['cleaned_2'].str.replace('\d+', '')
data['cleaned_3'].head(10)

<ipython-input-35-381df97760c3>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['cleaned_3'] = data['cleaned_2'].str.replace('\d+', '')
<ipython-input-35-381df97760c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_3'] = data['cleaned_2'].str.replace('\d+', '')


0    حدد رءيس الوزراء البريطاني ديفيد كاميرون اليوم...
1    اصدر الرءيس عبدربه منصور هادي رءيس الجمهوريه ا...
2    وافق مجلس الامن الدولي اليوم الاثنين علي اتفاق...
3    قال مسءولون افغان ان  جنديا افغانيا قتلوا حادث...
4    اعلنت وزاره الخارجيه الايطاليه اليوم الاثنين ا...
5    ارتكبت الميليشيات الحوثيه مجزره مدينه عدن جنوب...
6    تحد جديد للكونغرس الاميركي رفض الرءيس باراك او...
7    افاق سكان حي الشيخ رضوان وشارع النفق وسط مدينه...
8    تحول «فقهاء» وقاده تنظيم داعش ليبيا الي نوع جد...
9    وسط تفاقم ازمات الكهرباء والماء السودان لوحت ح...
Name: content, dtype: object